In [1]:
import numpy as np
import array
import sys
from PIL import Image
import tifffile

In [2]:
fileName = 'Camera_22_34_10'
inputFile = fileName + '.dat'
outputFile = fileName + '.tif'

In [3]:
with open(inputFile, 'rb') as fobj:
    raw_bytes = fobj.read()

In [10]:
# Find data tag for recording speed by searching
offset = raw_bytes.find(b'\x30\x30\x07\x30')
offset += 14 # account for the ten bytes of metadata (see below)
nbytes = 4
byte_data=raw_bytes[offset:nbytes+offset]

print('Recording speed: ' + byte_data.decode('ascii') + ' ns')

Recording speed: 1420 ns


In [11]:
# Find data tag for exposure time by searching
offset = raw_bytes.find(b'\x30\x30\x08\x30')
offset += 14 # account for the 14 bytes of metadata 

# Next 4 bytes are the actual value
nbytes = 4
byte_data=raw_bytes[offset:nbytes+offset]

print('Exposure time: ' + byte_data.decode('ascii') + ' ns')

Exposure time: 570  ns


In [12]:
# Find data tag for time of recording
offset = raw_bytes.find(b'\x40\x40\x0e\x40')
offset += 14 # account for the fourteen bytes of metadata 

# Next 16 bytes are the actual value
#offset += nbytes
nbytes = 16
byte_data=raw_bytes[offset:nbytes+offset]
#print(''.join(['\\x%02x' % b for b in byte_data]))
#print(byte_data)
year = int.from_bytes(byte_data[0:2],byteorder = "little", signed=False)
month = int.from_bytes(byte_data[2:4],byteorder = "little", signed=False)
day = int.from_bytes(byte_data[6:8],byteorder = "little", signed=False)
hour = int.from_bytes( byte_data[8:10], byteorder = "little", signed=False )
minute = int.from_bytes( byte_data[10:12], byteorder = "little", signed=False )
second = int.from_bytes( byte_data[12:14], byteorder = "little", signed=False )
print('Date: ' + str(year) + '-' + str(month) + '-' + str(day) )
print('Time: ' + str(hour) + ':' + str(minute) + ':' + str(second) )

Date: 2023-6-29
Time: 22:34:10


In [14]:
# Find data tag for image data
offset = raw_bytes.find(b'\xa0\xa0\x01\xa0')
offset += 14  # account for 14 bytes of metadata

# Next imageNum x 250 x 400 x 2 bytes are the actual values
imageNum = 128
nbytes = imageNum * 250 * 400 * 2
byte_data=raw_bytes[offset:nbytes+offset]

In [15]:
with tifffile.TiffWriter(outputFile) as tiff:
    for i in range(0,128):
        imagebytes=np.frombuffer(byte_data[i*200000:200000*(i+1)],dtype=np.int16)
        imagebytes=np.reshape(imagebytes,newshape=(250,400)) >> 6
        #frames.append(Image.fromarray(imagebytes,'I;16'))
    
        tiff.save(imagebytes)